# Машинное обучение, ФКН ВШЭ

## Практическое задание 8. Метод опорных векторов и аппроксимация ядер

### Общая информация

Дата выдачи: 30.01.2025

Мягкий дедлайн: 23:59MSK 16.02.2025

Жесткий дедлайн: 23:59MSK 23.02.2025

### Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимальная оценка за работу (без учёта бонусов) — 10 баллов.

Сдавать задание после указанного жёсткого срока сдачи нельзя. При выставлении неполного балла за задание в связи с наличием ошибок на усмотрение проверяющего предусмотрена возможность исправить работу на указанных в ответном письме условиях.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Использование генеративных языковых моделей разрешено только в случае явного указания на это. Необходимо прописать (в соответствующих пунктах, где использовались, либо в начале/конце работы):
- какая языковая модель использовалась
- какие использовались промпты и в каких частях работы
- с какими сложностями вы столкнулись при использовании генеративных моделей, с чем они помогли больше всего

Неэффективная реализация кода может негативно отразиться на оценке.

### Формат сдачи
Задания сдаются через систему anytask. Посылка должна содержать:
* Ноутбук homework-practice-08-random-features-Username.ipynb

Username — ваша фамилия и имя на латинице именно в таком порядке

### О задании

На занятиях мы подробно обсуждали метод опорных векторов (SVM). В базовой версии в нём нет чего-то особенного — мы всего лишь используем специальную функцию потерь, которая не требует устремлять отступы к бесконечности; ей достаточно, чтобы отступы были не меньше +1. Затем мы узнали, что SVM можно переписать в двойственном виде, который, позволяет заменить скалярные произведения объектов на ядра. Это будет соответствовать построению модели в новом пространстве более высокой размерности, координаты которого представляют собой нелинейные модификации исходных признаков.

Ядровой SVM, к сожалению, довольно затратен по памяти (нужно хранить матрицу Грама размера $d \times d$) и по времени (нужно решать задачу условной оптимизации с квадратичной функцией, а это не очень быстро). Мы обсуждали, что есть способы посчитать новые признаки $\tilde \varphi(x)$ на основе исходных так, что скалярные произведения этих новых $\langle \tilde \varphi(x), \tilde \varphi(z) \rangle$ приближают ядро $K(x, z)$.

Мы будем исследовать аппроксимации методом Random Fourier Features (RFF, также в литературе встречается название Random Kitchen Sinks) для гауссовых ядер. Будем использовать формулы, которые немного отличаются от того, что было на лекциях (мы добавим сдвиги внутрь тригонометрических функций и будем использовать только косинусы, потому что с нужным сдвигом косинус превратится в синус):
$$\tilde \varphi(x) = (
\cos (w_1^T x + b_1),
\dots,
\cos (w_n^T x + b_n)
),$$
где $w_j \sim \mathcal{N}(0, 1/\sigma^2)$, $b_j \sim U[-\pi, \pi]$.

На новых признаках $\tilde \varphi(x)$ мы будем строить любую линейную модель.

Можно считать, что это некоторая новая парадигма построения сложных моделей. Можно направленно искать сложные нелинейные закономерности в данных с помощью градиентного бустинга или нейронных сетей, а можно просто нагенерировать большое количество случайных нелинейных признаков и надеяться, что быстрая и простая модель (то есть линейная) сможет показать на них хорошее качество. В этом задании мы изучим, насколько работоспособна такая идея.

### Алгоритм

Вам потребуется реализовать следующий алгоритм:
1. Понизить размерность выборки до new_dim с помощью метода главных компонент.
2. Для полученной выборки оценить гиперпараметр $\sigma^2$ с помощью эвристики (рекомендуем считать медиану не по всем парам объектов, а по случайному подмножеству из где-то миллиона пар объектов): $$\sigma^2 = \text{median}_{i, j = 1, \dots, \ell, i \neq j} \left\{\sum_{k = 1}^{d} (x_{ik} - x_{jk})^2 \right\}$$
3. Сгенерировать n_features наборов весов $w_j$ и сдвигов $b_j$.
4. Сформировать n_features новых признаков по формулам, приведённым выше.
5. Обучить линейную модель (логистическую регрессию или SVM) на новых признаках.
6. Повторить преобразования (PCA, формирование новых признаков) к тестовой выборке и применить модель.

In [ ]:
%load_ext autoreload
%autoreload 2

Тестировать алгоритм мы будем на данных Fashion MNIST. Ниже код для их загрузки и подготовки.

In [ ]:
import numpy as np
import time
from sklearn.metrics import mean_squared_error as mse

# 1 Способ
import keras
from keras.datasets import fashion_mnist
(x_train_pics, y_train), (x_test_pics, y_test) = fashion_mnist.load_data()

# 2 Способ (если первый не работает)
# from sklearn.datasets import fetch_openml
# def load_fashion_mnist():
#     X, y = fetch_openml('Fashion-MNIST', version=1, return_X_y=True, as_frame=False)
#     X = X.reshape(-1, 28, 28).astype('uint8')
#     y = y.astype('int64')
#     x_train, x_test = X[:60000], X[60000:]
#     y_train, y_test = y[:60000], y[60000:]
#     return (x_train, y_train), (x_test, y_test)
# (x_train_pics, y_train), (x_test_pics, y_test) = load_fashion_mnist()




X_train = x_train_pics.reshape(y_train.shape[0], -1)
X_test = x_test_pics.reshape(y_test.shape[0], -1)

__Задание 0. (0.25 баллов)__

**Вопрос:** зачем в алгоритме нужен метод главных компонент?

**Ответ:** Интуитивно я бы сказал по тому, что главные (сильные) признаки при объединении дадут хорошие новые признаки, а всякие полукровки с маглами вообще потеряют магию (вызовут переобучение). Потому мы сначала выдедим лучшие, а затем уже на меньшем множестве (еще и генерация быстрее будет происходить) создадим новые

__Задание 1. (3 балла)__

Реализуйте алгоритм, описанный выше. Можете воспользоваться шаблоном класса в `homework_practice_08_rff.py` (допишите его и исправьте несостыковки в классе пайплайна) или написать свой интерфейс.

Ваша реализация должна поддерживать следующие опции:
1. Возможность задавать значения гиперпараметров new_dim (по умолчанию 50) и n_features (по умолчанию 1000).
2. Возможность включать или выключать предварительное понижение размерности с помощью метода главных компонент.
3. Возможность выбирать тип линейной модели (логистическая регрессия или SVM с линейным ядром).

Протестируйте на данных Fashion MNIST, сформированных кодом выше. Если на тесте у вас получилась доля верных ответов не ниже 0.84 с гиперпараметрами по умолчанию, то вы всё сделали правильно.

In [ ]:
from homework_practice_08_rff import RFFPipeline, RandomFeatureCreator
from sklearn.metrics import accuracy_score

pipeline = RFFPipeline(n_features=1000, new_dim=50, feature_creator_class=RandomFeatureCreator)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.8613


Как видим, алгоритм перформит.

__Задание 2. (2.5 балла)__

Сравните подход со случайными признаками с обучением SVM на исходных признаках. Попробуйте вариант с обычным (линейным) SVM и с ядровым SVM. Ядровой SVM может очень долго обучаться, поэтому можно делать любые разумные вещи для ускорения: брать подмножество объектов из обучающей выборки, например.

Сравните подход со случайными признаками с вариантом, в котором вы понижаете размерность с помощью PCA и обучите градиентный бустинг. Используйте одну из реализаций CatBoost/LightGBM/XGBoost.

Сделайте выводы — насколько идея со случайными признаками работает? Сравните как с точки зрения качества, так и с точки зрения скорости обучения и применения.

In [ ]:
x_train, Y_train = X_train, y_train

In [ ]:
subsample = 7777
idx_sub = np.random.choice(np.arange(X_train.shape[0]), size=subsample, replace=False)
X_train = X_train[idx_sub]
y_train = y_train[idx_sub]

In [ ]:
# Your code here: (￣▽￣)/♫•*¨*•.¸¸♪
from xgboost import XGBClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.decomposition import PCA

start_time = time.time()
linear_svm = LinearSVC()
linear_svm.fit(X_train, y_train)
y_pred_linear = linear_svm.predict(X_test)
acc_linear = accuracy_score(y_test, y_pred_linear)
print(f"Linear SVM accuracy = {acc_linear:.4f}, time = {time.time()-start_time:.2f}")

Linear SVM accuracy = 0.7664, time = 398.81


/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [ ]:
start_time = time.time()
kernel_svm = SVC(kernel='rbf', gamma='scale')
kernel_svm.fit(X_train, y_train)
y_pred_kernel = kernel_svm.predict(X_test)
acc_kernel = accuracy_score(y_test, y_pred_kernel)
print(f"Kernel SVM accuracy = {acc_kernel:.4f}, time = {time.time()-start_time:.2f}")

Kernel SVM accuracy = 0.8487, time = 48.05


In [ ]:
X_train, y_train = x_train, Y_train

In [ ]:
start_time = time.time()
pca = PCA(n_components=50, random_state=42)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)
xgb = XGBClassifier(eval_metric='mlogloss', random_state=42)
xgb.fit(X_train_pca, y_train)
y_pred_xgb = xgb.predict(X_test_pca)
acc_xgb = accuracy_score(y_test, y_pred_xgb)
print(f"XGBoost: accuracy = {acc_xgb:.4f}, time = {time.time()-start_time:.2f}")

XGBoost: accuracy = 0.8711, time = 38.99


С лучшими парметрами (перебирал оптуной (убрал ее потому как дохрена долго работала) accuracy 0.8858)

    n_components: 98
    max_depth: 6
    learning_rate: 0.1913593150564032
    subsample: 0.6269101897488234
    colsample_bytree: 0.8380798840530911
    n_estimators: 290

Как видим, бустинг, даже без подбора параметров, показывает результат лучше, чем SVM, причем как по акураси, так и по времени. Обычный SVM работал дольше всех, так еще и качество не очень, его модификация стала работать быстрее, чем самописный rff (48 секунд против 60), но и качество немного просело (0,8487 против 0,86). В целом можно говорить о том, что бустинг работает надежнее всех (по крайней мере в такой незамысловатой задаче).

__Задание 3. (2 балла)__

Проведите эксперименты:
1. Помогает ли предварительное понижение размерности с помощью PCA?
2. Как зависит итоговое качество от n_features? Выходит ли оно на плато при росте n_features?
3. Важно ли, какую модель обучать — логистическую регрессию или SVM?

In [ ]:
# Your code here: (￣▽￣)/♫•*¨*•.¸¸♪
start_time = time.time()
rff_pipe = RFFPipeline(n_features=1000, new_dim=50, use_PCA=True,
    feature_creator_class=RandomFeatureCreator)
rff_pipe.fit(X_train, y_train)
y_pred_rff = rff_pipe.predict(X_test)
acc_rff = accuracy_score(y_test, y_pred_rff)
print(f"RFF accuracy = {acc_rff:.4f}, time = {time.time()-start_time:.2f}")

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


RFF accuracy = 0.8616, time = 80.50


In [ ]:
start_time = time.time()
pipeline = RFFPipeline(n_features=1000, new_dim=50, feature_creator_class=RandomFeatureCreator)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy = {acc_rff:.4f}, time = {time.time()-start_time:.2f}")

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy = 0.8616, time = 62.37


В целом (смотря еще на дальнейшие эксперименты), я бы сказал, что PCA помогает улучшить метрики (причем очень существенно, например для Лапласа). При этом, при базовых параметрах эффект не особо виден.

In [ ]:
n_features_values = [50, 100, 200, 500, 1500]
for n_feat in n_features_values:
    pipeline = RFFPipeline(n_features=1000, new_dim=50, feature_creator_class=RandomFeatureCreator)
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"n_features = {n_feat}: accuracy = {acc:.4f}")

Видим, что если использовать PCA, то при росте n_features качество растет и дейстивтельно выходит на плато при дальнейшем увеличении (результат ниже, связано это с тем, что мы из большего набора способны выбрать лучший результат).

    n_features = 50: accuracy = 0.7749
    n_features = 100: accuracy = 0.8081
    n_features = 200: accuracy = 0.8346
    n_features = 500: accuracy = 0.8565
    n_features = 1500: accuracy = 0.8646

Если же PCA не использовать, то прироста это не дает (результаты в таком формате, т.к. запускал я не убрав ворнинги, а перезапускать долго).

    n_features = 50: accuracy = 0.8640
    n_features = 100: accuracy = 0.8622
    n_features = 200: accuracy = 0.8630
    n_features = 500: accuracy = 0.8619
    n_features = 1500: accuracy = 0.8633

In [ ]:
pipe_lr = RFFPipeline(
    n_features=1000,
    new_dim=50,
    use_PCA=True,
    feature_creator_class=RandomFeatureCreator,
    classifier_params={'max_iter': 1000, 'random_state': 42},
    func=np.cos
)
pipe_lr.fit(X_train, y_train)
acc_lr = accuracy_score(y_test, pipe_lr.predict(X_test))

pipe_svm = RFFPipeline(
    n_features=1000,
    new_dim=50,
    use_PCA=True,
    feature_creator_class=RandomFeatureCreator,
    classifier_class=LinearSVC,
    classifier_params={'max_iter': 10000, 'random_state': 42},
    func=np.cos
)
pipe_svm.fit(X_train, y_train)
acc_svm = accuracy_score(y_test, pipe_svm.predict(X_test))

print(f"LogisticRegression accuracy = {acc_lr:.4f}")
print(f"LinearSVC accuracy = {acc_svm:.4f}")

LogisticRegression accuracy = 0.8627
LinearSVC accuracy = 0.8694


Видно, что качество отличается не сильно, однако т.к. метод у нас заточен под SVM, да и нашей метрике побоку веротяности, SVM все же перформит лучше.

__Задание 4. (Максимум 1.5 балла)__

Как вы, должно быть, помните с курса МО-1, многие алгоритмы машинного обучения работают лучше, если признаки данных некоррелированы. Оказывается, что для RFF существует модификация, позволяющая получать ортогональные случайные признаки (Orthogonal Random Features, ORF). Об этом методе можно прочитать в [статье](https://proceedings.neurips.cc/paper/2016/file/53adaf494dc89ef7196d73636eb2451b-Paper.pdf). Реализуйте класс для вычисления ORF по аналогии с основным заданием. Обратите внимание, что ваш класс должен уметь работать со случаем n_features > new_dim (в статье есть замечание на этот счет), n_features=new_dim и n_features < new_dim также должны работать, убедитесь в этом. Проведите эксперименты, сравнивающие RFF и ORF, сделайте выводы.


In [ ]:
# Your code here: (￣▽￣)/♫•*¨*•.¸¸♪
from homework_practice_08_rff import OrthogonalRandomFeatureCreator
start_time = time.time()
rff_pipe = RFFPipeline(n_features=1000, new_dim=50, use_PCA=True,
    feature_creator_class=OrthogonalRandomFeatureCreator)
rff_pipe.fit(X_train, y_train)
y_pred_rff = rff_pipe.predict(X_test)
acc_rff = accuracy_score(y_test, y_pred_rff)
print(f"RFF accuracy = {acc_rff:.4f}, time = {time.time()-start_time:.2f}")

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


RFF accuracy = 0.8635, time = 65.10


Как видим, ORF работает чуть лучше, чем RFF (если поиграть с параметрами, можно еще пару пунктов выжать). По времени работает немного быстрее, да и видимо более численно устойчив (из-за ортогонализации).

__Задание 5. (Максимум 1 балл)__

Существует большое количество работ, где идея RFF развивается, предлагаются её обобщения (которые, по сути, выливаются в другие преобразования признаков, не обязательно уже тригонометрические). Возьмите любую из таких работ, кратко опишите идею, имплементируйте её и сравните качество с ORF и RFF, которые вы запрограммировали выше.

Ссылки на статьи, где обсуждаются вариации RFF для разных ядер, можно найти в окрестности таблицы 1 в работе https://arxiv.org/pdf/1407.5599  

___ссылка на работу:___ https://chbrown.github.io/kdd-2013-usb/kdd/p239.pdf

___описание идеи:___ Мы решаем аппроксимировать ядро $(<x,y> + c)^p$ через быстрое вычисление Count Sketch (по-сути это свертка вектора используя хэш-функции на индексы и ±1) а также fft для быстрого подсчета.

Я пробовал еще QMC, Лапласа и Коши. QMC показал неплохой результат, Лаплас работал плохо, Коши работал хорошо

In [ ]:
# Your code here: (￣▽￣)/♫•*¨*•.¸¸♪
from homework_practice_08_rff import TensorSketchFeatureCreator
start_time = time.time()
rff_pipe = RFFPipeline(n_features=1000, new_dim=50, use_PCA=True,
    feature_creator_class=TensorSketchFeatureCreator)
rff_pipe.fit(X_train, y_train)
y_pred_rff = rff_pipe.predict(X_test)
acc_rff = accuracy_score(y_test, y_pred_rff)
print(f"RFF accuracy = {acc_rff:.4f}, time = {time.time()-start_time:.2f}")

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


RFF accuracy = 0.8604, time = 201.14


Видим, что метод стал работаь медленнее, однако качество остается достаточно высоким и сравнимы как с RFF, так и с ORF.

__Задание 6. (Максимум 2.5 балла)__

Реализуйте класс ядровой Ridge регрессии (Лекция 13, $\S 1.2$), для оптимизации используте градиентный спуск **[1 балл максимум]**, также добавьте возможность использовать аналитическую формулу **[1 балл максимум]**. Для градиентного спуска выпишите градиент ниже **[0.5 баллов максимум]**.
Подумайте о том, как в формулах правильно учесть свободный коэффициент.

Затем адаптируйте вашу реализацию RFF под задачу регрессии. Сравните вашу ядровую регрессию и RFF на синтетических данных.

Функция потерь:
$$
Q(w) = \frac{1}{2} ||\Phi \Phi^T w - y||^2 + \frac{\lambda}{2} w^T \Phi \Phi^T w \rightarrow \min_w,
$$
где $\Phi \Phi^T = K$, $K = (k(x_i, x_j))_{i, j = 1}^{\ell}$.

Предсказание:
$
y(x) = k(x)^T w,
$
где $k(x)$ — вектор функций ядра от пар объектов $(x, x_i)_{i=1}^{\ell}$.

___Выведите градиент:___
$$
\nabla Q(w) \implies d\frac{(y^t - w^tK)(Kw - y) + λw^tKw}{2} = \frac{dw^t(λKw - K(Kw-y)) + ((y^t - w^tK)K + λw^tK)dw}{2} = |стоит\ скаляр\ потому\ транспонируем| = (K(Kw-y) + λKw)^tdw \implies \nabla Q(w) = K(Kw- y + λw)
$$

Вы можете изменять представленный шаблон в файле `homework_practice_08_kernel_regression.py` по своему усмотрению.

In [ ]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


X, y = make_regression(n_samples=3000, n_features=100, noise=0.1)
ss = StandardScaler()
X = ss.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
from homework_practice_08_kernel_regression import KernelRidgeRegression

start_time = time.time()
kernel_linreg = KernelRidgeRegression().fit_closed_form(X_train, y_train)
y_pred_reg = kernel_linreg.predict(X_test)
acc = mse(y_test, y_pred_reg)
print(f"MSE = {acc:.4f}, time = {time.time()-start_time:.2f}")

MSE = 22812.5675, time = 0.67


In [ ]:
from homework_practice_08_rff import RFFPipeline, RandomFeatureCreator

pipeline = RFFPipeline(
    n_features=250,
    feature_creator_class=RandomFeatureCreator,
    regression=True
)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
accuracy = mse(y_test, y_pred)
print("MSE:", accuracy)

MSE: 3016.3979959673647


Ну я думаю тут все и без слов понятно... Возможно, надо какие-то оболее адекватные данные, но мы видим, что даже без PCA RFF показывает существенно лучший резульат.